In [5]:
import csv
import gc
import json
import pickle
import os

import pandas as pd
import numpy as np
import tensorflow as tf
import tf2onnx

## Откроем датасет https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package

In [2]:
with open('datasets/weatherAUS.csv') as f:
    reader = csv.reader(f)
    _data = list(reader)
    df = pd.DataFrame(_data[1:], columns=_data[0])

In [3]:
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NA,NA,W,44,W,...,71,22,1007.7,1007.1,8,NA,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0,NA,NA,WNW,44,NNW,...,44,25,1010.6,1007.8,NA,NA,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0,NA,NA,WSW,46,W,...,38,30,1007.6,1008.7,NA,2,21,23.2,No,No
3,2008-12-04,Albury,9.2,28,0,NA,NA,NE,24,SE,...,45,16,1017.6,1012.8,NA,NA,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1,NA,NA,W,41,ENE,...,82,33,1010.8,1006,7,8,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.4,0,NA,NA,E,31,SE,...,51,24,1024.6,1020.3,NA,NA,10.1,22.4,No,No
145456,2017-06-22,Uluru,3.6,25.3,0,NA,NA,NNW,22,SE,...,56,21,1023.5,1019.1,NA,NA,10.9,24.5,No,No
145457,2017-06-23,Uluru,5.4,26.9,0,NA,NA,N,37,SE,...,53,24,1021,1016.8,NA,NA,12.5,26.1,No,No
145458,2017-06-24,Uluru,7.8,27,0,NA,NA,SE,28,SSE,...,51,24,1019.4,1016.5,3,2,15.1,26,No,No


In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(8,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [6]:
# Берем только числовые колонки
columns = ['RainToday', 'MinTemp', 'MaxTemp', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm']
# Убираем пустые значения
df = df[df[columns]!='NA'].dropna(subset=columns)
# Делим на X, Y
X, Y = df.loc[:, df.columns!='RainToday'], df.loc[:, df.columns=='RainToday']
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,NaN,NaN,13.4,22.9,NaN,NaN,NaN,NaN,NaN,NaN,...,71,22,1007.7,1007.1,NaN,NaN,16.9,21.8,No,NaN
1,NaN,NaN,7.4,25.1,NaN,NaN,NaN,NaN,NaN,NaN,...,44,25,1010.6,1007.8,NaN,NaN,17.2,24.3,No,NaN
2,NaN,NaN,12.9,25.7,NaN,NaN,NaN,NaN,NaN,NaN,...,38,30,1007.6,1008.7,NaN,NaN,21,23.2,No,NaN
3,NaN,NaN,9.2,28,NaN,NaN,NaN,NaN,NaN,NaN,...,45,16,1017.6,1012.8,NaN,NaN,18.1,26.5,No,NaN
4,NaN,NaN,17.5,32.3,NaN,NaN,NaN,NaN,NaN,NaN,...,82,33,1010.8,1006,NaN,NaN,17.8,29.7,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145454,NaN,NaN,3.5,21.8,NaN,NaN,NaN,NaN,NaN,NaN,...,59,27,1024.7,1021.2,NaN,NaN,9.4,20.9,No,NaN
145455,NaN,NaN,2.8,23.4,NaN,NaN,NaN,NaN,NaN,NaN,...,51,24,1024.6,1020.3,NaN,NaN,10.1,22.4,No,NaN
145456,NaN,NaN,3.6,25.3,NaN,NaN,NaN,NaN,NaN,NaN,...,56,21,1023.5,1019.1,NaN,NaN,10.9,24.5,No,NaN
145457,NaN,NaN,5.4,26.9,NaN,NaN,NaN,NaN,NaN,NaN,...,53,24,1021,1016.8,NaN,NaN,12.5,26.1,No,NaN


In [7]:
# Приготовим данные, приведем X в float64, а Y в int64
X[columns[1:]] = X[columns[1:]].astype({col: float for col in columns[1:]})
Y = Y.RainToday.map(dict(Yes=1, No=0))

/var/folders/bg/6fjjsrn55s3g6s3781w40rtc0000gq/T/ipykernel_13666/3023816294.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[columns[1:]] = X[columns[1:]].astype({col: float for col in columns[1:]})


In [8]:
model.fit(X[columns[1:]], Y, epochs=5)

Epoch 1/5
3918/3918 [==============================] - 6s 1ms/step - loss: 0.8959 - accuracy: 0.7489
Epoch 2/5
3918/3918 [==============================] - 5s 1ms/step - loss: 0.6359 - accuracy: 0.7687
Epoch 3/5
3918/3918 [==============================] - 5s 1ms/step - loss: 0.5240 - accuracy: 0.7805
Epoch 4/5
3918/3918 [==============================] - 4s 1ms/step - loss: 0.4713 - accuracy: 0.7901
Epoch 5/5
3918/3918 [==============================] - 6s 1ms/step - loss: 0.4371 - accuracy: 0.7989


In [10]:
model(X[columns[1:]].values)

<tf.Tensor: shape=(125347, 1), dtype=float32, numpy=
array([[0.48739007],
       [0.03589731],
       [0.09037315],
       ...,
       [0.02889997],
       [0.02520086],
       [0.03715603]], dtype=float32)>

In [11]:
model.__call__(X[columns[1:]].values)

<tf.Tensor: shape=(125347, 1), dtype=float32, numpy=
array([[0.48739007],
       [0.03589731],
       [0.09037315],
       ...,
       [0.02889997],
       [0.02520086],
       [0.03715603]], dtype=float32)>

In [12]:
model(np.array([[1., 2., 3., 4., 5., 6., 7., 8.]]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.51441157]], dtype=float32)>

## Сохраняем модель

In [13]:
with open("models/tensorflow-binary_cls_model.pckl", "wb") as f:
    pickle.dump(model, f)

    
model.save("models/tensorflow-binary_cls_model.keras")
model.save("models/tensorflow-binary_cls_model.h5")
model.export("models/tensorflow-binary_cls_model.savedmodel")

INFO:tensorflow:Assets written to: models/tensorflow-binary_cls_model.savedmodel/assets


/Users/t.deys/.pyenv/versions/3.10.11/envs/mlup-ws/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
INFO:tensorflow:Assets written to: models/tensorflow-binary_cls_model.savedmodel/assets


Saved artifact at 'models/tensorflow-binary_cls_model.savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  Args:
    args_0: float32 Tensor, shape=(None, 8)
  Returns:
    float32 Tensor, shape=(None, 1)


In [35]:
model.graph._captures = 
model_proto, _ = tf2onnx.convert.from_keras(
    model, 
    input_signature=(tf.TensorSpec((None, 8), tf.float32, name="input"),), 
    opset=10, 
    output_path="models/tensorflow-binary_cls_model.onnx"
)
output_names = [n.name for n in model_proto.graph.output]

AttributeError: 'FuncGraph' object has no attribute '_captures'

In [18]:
tf.keras.models.model_from_json(j)

## Columns
* 'MinTemp'
* 'MaxTemp'
* 'Humidity9am'
* 'Humidity3pm'
* 'Pressure9am'
* 'Pressure3pm'
* 'Temp9am'
* 'Temp3pm'

Принимает на вход массив из 8 float64 числа. Выдает тензор с числом

## Requirements
* tensorflow
* pandas